In [1]:
# pip install cleantext

In [2]:
import numpy as np
import pandas as pd
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from cleantext import clean
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sarathkumar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sarathkumar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sarathkumar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\sarathkumar\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
# Set file path
input_file_path = 'D:\\TCD\\Term1\\CS7CS4 - ML\\group project\\MachineLearning-group-project\\RawDatasets\\'

In [4]:
# Read the data set
df = pd.read_csv(input_file_path + 'tweets_india.csv', usecols=['content'])
# df['Origin'] = 'southasia'

In [5]:
print('-------------------')
print('Dataset content')
print('-------------------')
print(df.head())
print('-------------------')
# Some information about the data set
print('Length of data set: ', len(df))
print('Shape of data set: ', df.shape)

print('-------------------')
print('Dataset information')
print('-------------------')
df.info()

-------------------
Dataset content
-------------------
                                             content
0  Have a nice night trip with T \r\nUK to Delhi....
1  May her soul RIP. As a nation we are with the ...
2  @SanjayAzadSln अत्यंत दुखद दिवंगत आत्मा को श्र...
3  @firki07 Firki with queen 😭 https://t.co/mhvgH...
4  @TuckerCarlson This so called great Britain lo...
-------------------
Length of data set:  932
Shape of data set:  (932, 1)
-------------------
Dataset information
-------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 932 entries, 0 to 931
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  932 non-null    object
dtypes: object(1)
memory usage: 7.4+ KB


In [6]:
# Check for null values - needs to be zero
np.sum(df.isnull().any(axis=1))

0

In [7]:
# 1. Converting all text to lower case
def convert_to_lower_case(data_set):
    data_set['content'] = data_set['content'].str.lower()

convert_to_lower_case(df)
df.head()

,content
0,have a nice night trip with t \r\nuk to delhi....
1,may her soul rip. as a nation we are with the ...
2,@sanjayazadsln अत्यंत दुखद दिवंगत आत्मा को श्र...
3,@firki07 firki with queen 😭 https://t.co/mhvgh...
4,@tuckercarlson this so called great britain lo...


In [8]:
# 2. Removing stop words (un-necessary words) - using nltk's pre-defined stop words
STOP_WORDS = set(stopwords.words('english'))
print('Stop word list:')
print('----------------')
print(STOP_WORDS)

def remove_stop_words(content):
   return " ".join([text for text in str(content).split() if text not in STOP_WORDS])

df['content'] = df['content'].apply(lambda content: remove_stop_words(content=content))
df.head()

Stop word list:
----------------
{'myself', 'herself', "it's", 'very', 'won', 'over', 'those', 'be', "couldn't", 'between', "you'll", 's', 'both', 'he', 'hers', 'out', 'haven', 'being', "doesn't", "isn't", 'can', 'with', 'about', "should've", 'am', 'which', 'are', 'had', "don't", "she's", 'where', 'shan', 'they', 'the', 'some', 'it', 'only', 'by', 'off', 'she', 'theirs', "you've", 'himself', 'not', "hadn't", 've', 'wouldn', 'will', 'i', 'shouldn', 'below', 'what', 'didn', 'hadn', 'yours', 'any', 'there', "that'll", 'on', 'his', 'all', 'themselves', 'further', 'than', 'into', 'her', 'our', 'm', 'through', 'for', 'isn', 'no', 'against', 'ours', 'we', "shan't", 'mightn', 'each', 'and', 'll', 'don', 'at', 'why', "mightn't", "shouldn't", 'here', 'too', 'my', 'yourself', "mustn't", 'do', 'their', 'during', 'other', 'weren', 'him', 'once', 'were', 'this', 'while', 'own', 'ain', 'them', 'been', 'then', 'itself', 'did', 'up', 'aren', 'mustn', 'doing', 'ma', 're', 'wasn', 'or', 'but', "hasn't", 

,content
0,nice night trip uk delhi. 😍😍😍 https://t.co/7am...
1,may soul rip. nation family fellow uk citizens...
2,@sanjayazadsln अत्यंत दुखद दिवंगत आत्मा को श्र...
3,@firki07 firki queen 😭 https://t.co/mhvgh7aqsc
4,@tuckercarlson called great britain looted 45 ...


In [9]:
# 3. Removing URLs
def remove_URLS(content):
    return re.sub('((www.[^s]+)|(https?://[^s]+))',' ', str(content))

df['content'] = df['content'].apply(lambda content: remove_URLS(content=content))
#df['content'].head()
df.head()

,content
0,nice night trip uk delhi. 😍😍😍
1,may soul rip. nation family fellow uk citizens...
2,@sanjayazadsln अत्यंत दुखद दिवंगत आत्मा को श्र...
3,@firki07 firki queen 😭 sc
4,@tuckercarlson called great britain looted 45 ...


In [10]:
# 4. Removing @mentions
def remove_mentions(content):
    return re.sub('(@\S+)',' ', str(content))

df['content'] = df['content'].apply(lambda content: remove_mentions(content=content))
#df['content'].head()
df.head()

,content
0,nice night trip uk delhi. 😍😍😍
1,may soul rip. nation family fellow uk citizens...
2,अत्यंत दुखद दिवंगत आत्मा को श्री चरणों में उ...
3,firki queen 😭 sc
4,called great britain looted 45 trillion usd ...


In [11]:
# 5. Removing numbers
def remove_numericals(content):
        return re.sub('[0-9]+', '', content)

df['content'] = df['content'].apply(lambda content: remove_numericals(content=content))
#df['content'].head()
df.head()

,content
0,nice night trip uk delhi. 😍😍😍
1,may soul rip. nation family fellow uk citizens...
2,अत्यंत दुखद दिवंगत आत्मा को श्री चरणों में उ...
3,firki queen 😭 sc
4,called great britain looted trillion usd in...


In [12]:
# 6. Removing punctuations
PUNCTUATIONS = string.punctuation
print('Punctuation list:')
print('------------------')
print(PUNCTUATIONS)

def remove_punctuations(content):
    return str(content).translate(str.maketrans('', '', PUNCTUATIONS))

df['content'] = df['content'].apply(lambda content: remove_punctuations(content=content))
# df['content'].head()
df.head()

Punctuation list:
------------------
!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


,content
0,nice night trip uk delhi 😍😍😍
1,may soul rip nation family fellow uk citizens ...
2,अत्यंत दुखद दिवंगत आत्मा को श्री चरणों में उ...
3,firki queen 😭 sc
4,called great britain looted trillion usd in...


In [13]:
# df[df.colA.map(lambda x: x.isascii())]

df1=df.copy()
# df1['DB_user'].str.encode('ascii', 'ignore').str.decode('ascii')
# df1['new'] = df1['content'].apply(lambda content: content.encode('ascii', 'ignore').str.decode('ascii'))
# df1.head()


In [14]:
# 7. Removing 2 letter random words, as it gives no meaning
def remove_2letter_words(content):
    w = w = re.sub(r'\b\w{1,2}\b', '', content)
    return re.sub(' +', ' ', w)

df['content'] = df['content'].apply(lambda content: remove_2letter_words(content=content))
df.head()

,content
0,nice night trip delhi 😍😍😍
1,may soul rip nation family fellow citizens mom...
2,्ं ु िं ्ा ो ्ी चरणों ें ् ्ा िे िा ो ी ु ेे ...
3,firki queen 😭
4,called great britain looted trillion usd india


In [15]:
# 8. convert emoji to text
def emoji_convert(content):
    return kp_all_emoji_emoticons.replace_keywords(content)

from emot.emo_unicode import UNICODE_EMOJI, UNICODE_EMOJI_ALIAS, EMOTICONS_EMO
from flashtext import KeywordProcessor
all_emoji_emoticons = {**EMOTICONS_EMO,**UNICODE_EMOJI_ALIAS, **UNICODE_EMOJI_ALIAS}
all_emoji_emoticons = {k:v.replace(":","").replace("_"," ").strip() for k,v in all_emoji_emoticons.items()}
kp_all_emoji_emoticons = KeywordProcessor()
for k,v in all_emoji_emoticons.items():
    kp_all_emoji_emoticons.add_keyword(k, v)

df['content'] = df['content'].apply(lambda content: emoji_convert(content=content))

df.head()

,content
0,nice night trip delhi heart eyes😍heart eyes
1,may soul rip nation family fellow citizens mom...
2,्ं ु िं ्ा ो ्ी चरणों ें ् ्ा िे िा ो ी ु ेे ...
3,firki queen sob
4,called great britain looted trillion usd india


In [16]:
# 9. Remove unwanted emojis
df['content'] = df['content'].apply(lambda content: clean(str(content), no_emoji=True))
df.head()

,content
0,nice night trip delhi heart eyesheart eyes
1,may soul rip nation family fellow citizens mom...
2,n u in aa o ii crnnon en aa ie iaa o ii u ee i...
3,firki queen sob
4,called great britain looted trillion usd india


In [22]:
# 10. Removing meaningless words
nltk.download('words')
words = set(nltk.corpus.words.words())
df['nonEnglish'] = df['content'].apply(lambda x: x.isascii() if isinstance(x, str) else False)
df = df[df['nonEnglish'] == True]
df = df.drop(['nonEnglish'], axis=1)
df['content'] = df['content'].apply(lambda content: remove_2letter_words(content=content))
df['content'] = df['content'].apply(lambda content: remove_stop_words(content=content))
df.head()

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\sarathkumar\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


,content
0,nice night trip delhi heart eyesheart eyes
1,may soul rip nation family fellow citizens mom...
2,crnnon iaa horrendous shame state acts happens...
3,firki queen sob
4,called great britain looted trillion usd india


In [ ]:
df.to_csv('D:\\TCD\\Term1\\CS7CS4 - ML\\group project\\MachineLearning-group-project\\CleanedDatasets\\SouthAsia_Cleaned.csv', index=False)